# MCP Database Tools

**Building E-commerce Database Tools with MCP**

---

Welcome to the world of **database-powered MCP servers**! This notebook shows you how to create sophisticated database tools that AI agents can use. By the end of this 10-minute tutorial, you'll have a fully functional e-commerce database accessible through MCP.

### 🎯 What You'll Learn

In this tutorial, you will:
- Create an SQLite in-memory database
- Build an e-commerce schema with sample data
- Expose database operations as MCP tools
- Query products, customers, and orders
- Handle complex SQL operations safely
- Build a shopping assistant agent

### 🛍️ Why E-commerce?

E-commerce is a perfect example because:
- **Universal concepts**: Products, orders, customers
- **Real-world relevance**: Everyone understands shopping
- **Rich operations**: Search, filter, analytics
- **Complex relationships**: Orders contain products from customers

## 📦 Step 1: Setting Up Dependencies

### Required Packages
We'll use SQLite (built into Python) for our database and MCP for the server.

### 📚 What We're Using
- **sqlite3**: Built-in Python database
- **mcp**: Model Context Protocol
- **strands**: Agent framework

In [ ]:
import sqlite3
import json
from datetime import datetime
from typing import List, Dict, Any, Optional

# MCP and Strands imports
from mcp.server import FastMCP
from strands import Agent
from strands.tools.mcp import MCPClient
from strands.models import BedrockModel
import boto3

print("✅ All dependencies ready!")
print("   SQLite version:", sqlite3.sqlite_version)
print("   Ready to build our e-commerce database! 🛍️")

## 🗄️ Step 2: Understanding the Database Schema

### E-commerce Data Model
Our database will have four main tables:

```mermaid
erDiagram
    PRODUCTS ||--o{ ORDER_ITEMS : contains
    CUSTOMERS ||--o{ ORDERS : places
    ORDERS ||--o{ ORDER_ITEMS : has
    
    PRODUCTS {
        int id PK
        string name
        string description
        float price
        int stock
        string category
    }
    
    CUSTOMERS {
        int id PK
        string name
        string email
        datetime created_at
    }
    
    ORDERS {
        int id PK
        int customer_id FK
        datetime order_date
        float total_amount
        string status
    }
    
    ORDER_ITEMS {
        int id PK
        int order_id FK
        int product_id FK
        int quantity
        float price
    }
```

## 🏗️ Step 3: Creating the Database

### In-Memory SQLite
We'll use `:memory:` to create a database that exists only while the program runs. Perfect for tutorials!

In [ ]:
def create_ecommerce_database():
    """Create and populate an in-memory e-commerce database."""
    conn = sqlite3.connect(':memory:')
    conn.row_factory = sqlite3.Row  # Enable column access by name
    
    # Create tables
    conn.executescript('''
        CREATE TABLE products (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            description TEXT,
            price REAL NOT NULL,
            stock INTEGER DEFAULT 0,
            category TEXT NOT NULL
        );
        
        CREATE TABLE customers (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            email TEXT UNIQUE NOT NULL,
            created_at TEXT DEFAULT CURRENT_TIMESTAMP
        );
        
        CREATE TABLE orders (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            customer_id INTEGER NOT NULL,
            order_date TEXT DEFAULT CURRENT_TIMESTAMP,
            total_amount REAL NOT NULL,
            status TEXT DEFAULT 'pending',
            FOREIGN KEY (customer_id) REFERENCES customers (id)
        );
        
        CREATE TABLE order_items (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            order_id INTEGER NOT NULL,
            product_id INTEGER NOT NULL,
            quantity INTEGER NOT NULL,
            price REAL NOT NULL,
            FOREIGN KEY (order_id) REFERENCES orders (id),
            FOREIGN KEY (product_id) REFERENCES products (id)
        );
    ''')
    
    print("✅ Database schema created!")
    return conn

# Create the database
conn = create_ecommerce_database()

## 📊 Step 4: Populating Sample Data

### Realistic E-commerce Data
Let's add products, customers, and orders to make our database useful.

In [ ]:
def populate_sample_data(conn):
    """Add sample e-commerce data to the database."""
    
    # Insert products
    products = [
        ('Laptop Pro 15"', 'High-performance laptop with 16GB RAM', 1299.99, 10, 'Electronics'),
        ('Wireless Mouse', 'Ergonomic wireless mouse with USB receiver', 29.99, 50, 'Electronics'),
        ('Mechanical Keyboard', 'RGB mechanical keyboard with blue switches', 89.99, 25, 'Electronics'),
        ('USB-C Hub', '7-in-1 USB-C hub with HDMI and card reader', 49.99, 30, 'Electronics'),
        ('Standing Desk', 'Electric height-adjustable standing desk', 399.99, 5, 'Furniture'),
        ('Ergonomic Chair', 'Mesh back office chair with lumbar support', 299.99, 8, 'Furniture'),
        ('Notebook Set', 'Pack of 3 premium notebooks', 19.99, 100, 'Stationery'),
        ('Premium Coffee Beans', 'Single-origin arabica coffee beans 1kg', 24.99, 40, 'Food & Drink'),
        ('Thermal Mug', 'Insulated stainless steel travel mug', 19.99, 60, 'Food & Drink'),
        ('Desk Lamp', 'LED desk lamp with adjustable brightness', 39.99, 20, 'Furniture')
    ]
    
    conn.executemany(
        'INSERT INTO products (name, description, price, stock, category) VALUES (?, ?, ?, ?, ?)',
        products
    )
    
    # Insert customers
    customers = [
        ('Alice Johnson', 'alice@example.com'),
        ('Bob Smith', 'bob@example.com'),
        ('Charlie Brown', 'charlie@example.com'),
        ('Diana Prince', 'diana@example.com')
    ]
    
    conn.executemany(
        'INSERT INTO customers (name, email) VALUES (?, ?)',
        customers
    )
    
    # Insert orders
    conn.execute('INSERT INTO orders (customer_id, total_amount, status) VALUES (1, 1329.98, "completed")')
    conn.execute('INSERT INTO orders (customer_id, total_amount, status) VALUES (2, 119.98, "completed")')
    conn.execute('INSERT INTO orders (customer_id, total_amount, status) VALUES (3, 39.98, "pending")')
    
    # Insert order items
    order_items = [
        (1, 1, 1, 1299.99),  # Order 1: 1 Laptop
        (1, 2, 1, 29.99),    # Order 1: 1 Mouse
        (2, 3, 1, 89.99),    # Order 2: 1 Keyboard
        (2, 2, 1, 29.99),    # Order 2: 1 Mouse
        (3, 7, 2, 19.99)     # Order 3: 2 Notebook Sets
    ]
    
    conn.executemany(
        'INSERT INTO order_items (order_id, product_id, quantity, price) VALUES (?, ?, ?, ?)',
        order_items
    )
    
    conn.commit()
    print("✅ Sample data populated!")
    
    # Show statistics
    stats = {}
    for table in ['products', 'customers', 'orders', 'order_items']:
        cursor = conn.execute(f'SELECT COUNT(*) FROM {table}')
        stats[table] = cursor.fetchone()[0]
    
    print("\n📊 Database Statistics:")
    for table, count in stats.items():
        print(f"   {table}: {count} records")

# Populate the database
populate_sample_data(conn)

## 🔧 Step 5: Creating Database MCP Tools

### Tool Categories
We'll create tools for:
1. **Product Search**: Find products by name or category
2. **Customer Management**: Look up customer information
3. **Order Processing**: View orders and details
4. **Analytics**: Sales statistics and insights

In [ ]:
# Create MCP server for database operations
db_mcp = FastMCP("E-commerce Database")

# Store connection globally for MCP tools
global_conn = conn

@db_mcp.tool(description="Search products by name or category")
def search_products(query: str = "", category: Optional[str] = None) -> List[Dict[str, Any]]:
    """Search for products in the database."""
    if category:
        cursor = global_conn.execute(
            "SELECT * FROM products WHERE category = ? AND name LIKE ?",
            (category, f"%{query}%")
        )
    else:
        cursor = global_conn.execute(
            "SELECT * FROM products WHERE name LIKE ? OR description LIKE ?",
            (f"%{query}%", f"%{query}%")
        )
    
    return [dict(row) for row in cursor.fetchall()]

@db_mcp.tool(description="Get customer information by email")
def get_customer(email: str) -> Optional[Dict[str, Any]]:
    """Get customer details by email address."""
    cursor = global_conn.execute(
        "SELECT * FROM customers WHERE email = ?", (email,)
    )
    row = cursor.fetchone()
    return dict(row) if row else None

@db_mcp.tool(description="Get order details including items")
def get_order_details(order_id: int) -> Optional[Dict[str, Any]]:
    """Get complete order information including all items."""
    # Get order info
    cursor = global_conn.execute("""
        SELECT o.*, c.name as customer_name, c.email
        FROM orders o
        JOIN customers c ON o.customer_id = c.id
        WHERE o.id = ?
    """, (order_id,))
    
    order = cursor.fetchone()
    if not order:
        return None
    
    order_dict = dict(order)
    
    # Get order items
    cursor = global_conn.execute("""
        SELECT oi.*, p.name as product_name, p.category
        FROM order_items oi
        JOIN products p ON oi.product_id = p.id
        WHERE oi.order_id = ?
    """, (order_id,))
    
    order_dict['items'] = [dict(row) for row in cursor.fetchall()]
    
    return order_dict

@db_mcp.tool(description="Get sales statistics by category")
def get_sales_by_category() -> List[Dict[str, Any]]:
    """Get total sales amount and quantity by product category."""
    cursor = global_conn.execute("""
        SELECT 
            p.category,
            COUNT(DISTINCT oi.order_id) as total_orders,
            SUM(oi.quantity) as total_quantity,
            SUM(oi.quantity * oi.price) as total_revenue
        FROM order_items oi
        JOIN products p ON oi.product_id = p.id
        JOIN orders o ON oi.order_id = o.id
        WHERE o.status = 'completed'
        GROUP BY p.category
        ORDER BY total_revenue DESC
    """)
    
    return [dict(row) for row in cursor.fetchall()]

@db_mcp.tool(description="Check product stock availability")
def check_stock(product_id: int) -> Dict[str, Any]:
    """Check if a product is in stock."""
    cursor = global_conn.execute(
        "SELECT id, name, stock, price FROM products WHERE id = ?", 
        (product_id,)
    )
    row = cursor.fetchone()
    
    if not row:
        return {"available": False, "error": "Product not found"}
    
    product = dict(row)
    product['available'] = product['stock'] > 0
    return product

print("🔧 Database MCP tools created!")
print("   Tools: search_products, get_customer, get_order_details,")
print("          get_sales_by_category, check_stock")

## 🤖 Step 6: Connecting to the Database Server

### Using the External Server
In production, the database server runs as a separate process. Let's connect to it!

In [ ]:
# Set up AWS Bedrock
session = boto3.Session(profile_name='default')
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    boto_session=session
)

# Create MCP client for the database server
from mcp import stdio_client, StdioServerParameters

db_mcp_client = MCPClient(lambda: stdio_client(
    StdioServerParameters(
        command="python",
        args=["../src/mcp_servers/ecommerce_db_server.py"]
    )
))

print("🔌 Database MCP client configured!")
print("   Server: E-commerce Database Server")
print("   Transport: stdio")

## 🛍️ Step 7: Creating a Shopping Assistant

### AI-Powered E-commerce
Let's create an agent that can help customers shop using our database tools.

In [ ]:
# Create shopping assistant agent
print("🤖 Creating shopping assistant...\n")

with db_mcp_client:
    # Get tools from the database server
    tools = db_mcp_client.list_tools_sync()
    
    print("📋 Available database tools:")
    for tool in tools:
        print(f"   - {tool.name}: {tool.description}")
    
    # Create the shopping assistant
    shopping_assistant = Agent(
        model=bedrock_model,
        system_prompt="""You are a helpful e-commerce shopping assistant.
        
        Your capabilities:
        - Search for products by name or category
        - Check product availability and stock
        - Look up customer information
        - Review order history and details
        - Provide sales analytics
        
        Always be helpful, accurate, and provide detailed information.
        Format prices nicely and mention stock levels when relevant.""",
        tools=tools
    )
    
    print("\n✅ Shopping assistant ready!")

## 💬 Step 8: Testing the Shopping Assistant

### Real E-commerce Queries
Let's test our assistant with various shopping scenarios.

In [ ]:
# Test 1: Product search
with db_mcp_client:
    tools = db_mcp_client.list_tools_sync()
    agent = Agent(model=bedrock_model, tools=tools)
    
    print("🔍 Test 1: Product Search")
    print("="*50)
    response = agent("What electronics do you have under $100?")
    print(f"🤖 Assistant: {response}")
    print("\n" + "="*50 + "\n")

In [ ]:
# Test 2: Customer lookup
with db_mcp_client:
    tools = db_mcp_client.list_tools_sync()
    agent = Agent(model=bedrock_model, tools=tools)
    
    print("🔍 Test 2: Customer Information")
    print("="*50)
    response = agent("Can you find information about the customer alice@example.com?")
    print(f"🤖 Assistant: {response}")
    print("\n" + "="*50 + "\n")

In [ ]:
# Test 3: Order details
with db_mcp_client:
    tools = db_mcp_client.list_tools_sync()
    agent = Agent(model=bedrock_model, tools=tools)
    
    print("🔍 Test 3: Order Details")
    print("="*50)
    response = agent("Show me the details of order #1")
    print(f"🤖 Assistant: {response}")
    print("\n" + "="*50 + "\n")

In [ ]:
# Test 4: Analytics
with db_mcp_client:
    tools = db_mcp_client.list_tools_sync()
    agent = Agent(model=bedrock_model, tools=tools)
    
    print("🔍 Test 4: Sales Analytics")
    print("="*50)
    response = agent("Which product category has the highest sales?")
    print(f"🤖 Assistant: {response}")
    print("\n" + "="*50 + "\n")

## 🔍 Step 9: Advanced Database Patterns

### Production Considerations
Here are important patterns for real-world database MCP servers:

In [ ]:
print("🔍 ADVANCED DATABASE PATTERNS")
print("=" * 60)

patterns = {
    "🔒 Security": [
        "Use parameterized queries (we did!)",
        "Validate all inputs",
        "Implement access control",
        "Never expose raw SQL execution"
    ],
    "🚀 Performance": [
        "Add indexes for common queries",
        "Use connection pooling",
        "Implement query result caching",
        "Paginate large result sets"
    ],
    "💾 Persistence": [
        "Use file-based SQLite for persistence",
        "Consider PostgreSQL for production",
        "Implement backup strategies",
        "Handle migrations properly"
    ],
    "🔧 Error Handling": [
        "Gracefully handle connection failures",
        "Return meaningful error messages",
        "Log database operations",
        "Implement retry logic"
    ]
}

for category, items in patterns.items():
    print(f"\n{category}")
    for item in items:
        print(f"   • {item}")

# Example: Creating indexes
print("\n\n📊 Example: Adding Indexes")
print("="*40)
print("""conn.execute('''
    CREATE INDEX idx_products_category ON products(category);
    CREATE INDEX idx_products_name ON products(name);
    CREATE INDEX idx_orders_customer ON orders(customer_id);
    CREATE INDEX idx_order_items_order ON order_items(order_id);
''')""")

## 🎉 Congratulations!

### 🏆 What You've Accomplished

In just 10 minutes, you've:
- ✅ Created an SQLite in-memory database
- ✅ Built a complete e-commerce schema
- ✅ Populated realistic sample data
- ✅ Exposed database operations as MCP tools
- ✅ Created a shopping assistant agent
- ✅ Tested complex database queries

### 🚀 What's Next?

Now that you've mastered database MCP servers, you can:
1. **Add More Features** - Inventory management, user authentication
2. **Use Real Databases** - PostgreSQL, MySQL, MongoDB
3. **Implement Caching** - Redis for performance
4. **Build APIs** - REST or GraphQL endpoints

### 💡 Key Takeaways

1. **SQLite = Perfect for Prototypes**: Zero setup, full SQL support
2. **Parameterized Queries**: Always use them for security
3. **Rich Tools**: Expose meaningful operations, not raw SQL
4. **Context Matters**: Keep MCP connections alive with `with`

### 📚 Resources

- [SQLite Documentation](https://sqlite.org/docs.html)
- [MCP Documentation](https://modelcontextprotocol.io)
- [SQL Security Best Practices](https://owasp.org/www-project-sql-injection/)

### 🌟 Challenge Yourself

Try enhancing the e-commerce database with:
- Shopping cart functionality
- Product reviews and ratings
- Inventory tracking with alerts
- Customer purchase history analysis

Happy building with database MCP servers! 🚀🤖✨